# Feature selection on Breast Caner Wisconsin (Classification Problem)

In [34]:
import pandas as pd
import numpy as np
import sys
sys.path.append("../src/UBayFS")

from UBaymodel import UBaymodel
from UBayconstraint import UBayconstraint

In [35]:
data = pd.read_csv("./data/data.csv")
labels = pd.read_csv("./data/labels.csv").replace(("M","B"),(0,1)).astype(int)

## Build UBaymodel

In [36]:
model = UBaymodel(data=data,
                 target = labels,
                 feat_names = data.columns,
                 weights = [0.01],
                 M = 100, random_state=10,
                 method=[ "mrmr"],
                 nr_features = 20)

100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 72.24it/s]


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 73.56it/s]


In [37]:
model.feat_names

Index(['mean.radius', 'mean.texture', 'mean.perimeter', 'mean.area',
       'mean.smoothness', 'mean.compactness', 'mean.concavity',
       'mean.concave.points', 'mean.symmetry', 'mean.fractal.dimension',
       'radius.error', 'texture.error', 'perimeter.error', 'area.error',
       'smoothness.error', 'compactness.error', 'concavity.error',
       'concave.points.error', 'symmetry.error', 'fractal.dimension.error',
       'worst.radius', 'worst.texture', 'worst.perimeter', 'worst.area',
       'worst.smoothness', 'worst.compactness', 'worst.concavity',
       'worst.concave.points', 'worst.symmetry', 'worst.fractal.dimension'],
      dtype='object')

In [38]:
model2 = UBaymodel(data=data,
                 target = labels,
                 feat_names = data.columns,
                 weights = [0.01],
                 M = 100, random_state=5,
                 method=["fisher", "mrmr"],
                  nr_features = 10)

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 61.07it/s]


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 62.15it/s]


In [39]:
model2.nr_features

10

In [40]:
model.constraints

[]

In [41]:
model.getOptim()

{'optim_method': 'GA', 'popsize': 100, 'maxiter': 100}

In [42]:
model.ncol

30

In [43]:
weights = np.tile(np.array([10,15,20,16,15,10,12,17,21,14]),3)
strength = 1
weights = weights * strength / np.sum(weights)
weights
#print(weights.shape)
model.setWeights(weights)
model2.setWeights(weights)

In [44]:
model.getWeights()

array([0.02222222, 0.03333333, 0.04444444, 0.03555556, 0.03333333,
       0.02222222, 0.02666667, 0.03777778, 0.04666667, 0.03111111,
       0.02222222, 0.03333333, 0.04444444, 0.03555556, 0.03333333,
       0.02222222, 0.02666667, 0.03777778, 0.04666667, 0.03111111,
       0.02222222, 0.03333333, 0.04444444, 0.03555556, 0.03333333,
       0.02222222, 0.02666667, 0.03777778, 0.04666667, 0.03111111])

In [45]:
constraints = UBayconstraint(rho=np.array([np.Inf, 0.1, 1, 1]), 
                             constraint_types=["max_size", "must_link", "cannot_link", "cannot_link","jjj"], 
                             constraint_vars=[10, [0,10,20], [0,9], [19,22,23]], 
                             num_elements=data.shape[1])

In [46]:
model.setConstraints(constraints)
model2.setConstraints(constraints)

In [47]:
model.setOptim(optim_method="GA", popsize=100, maxiter=200)
model2.setOptim(optim_method="GA", popsize=100, maxiter=200)

In [48]:
all(model.posteriorExpectation() == model2.posteriorExpectation())

False

In [49]:
all(model.weights == model2.weights)

True

In [50]:
model.getOptim()

{'optim_method': 'GA', 'popsize': 100, 'maxiter': 200}

In [51]:
a = model.train()
b = model2.train()

In [52]:
state = np.zeros(30)
state[[2,24,23]] = 1
model.evaluateFS(state=state)

{'cardinality': 3.0,
 'total utility': -0.85,
 'posterior feature utility': 0.15,
 'admissibility': 0.0,
 'number of violated constraints': 0,
 'average feature correlation': 0.449}

In [53]:
# same features selected?
a[1] == b[1]

False

# Block model

## Version 1 with block list

In [54]:
block_list = [np.arange(0,10), np.arange(10,20), np.arange(20,30)]

In [55]:
block_constraints = UBayconstraint(rho=np.array([1]), 
                             constraint_types=["max_size"], 
                             constraint_vars=[1], 
                             num_elements=len(block_list),
                                block_list = block_list)

In [56]:
block_constraints.A

array([[1., 1., 1.]])

In [57]:
block_constraints.b

array([1.])

In [58]:
block_constraints.rho

array([1.])

In [59]:
pd.DataFrame(block_constraints.block_matrix)

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


## Version 2 with block matrix

In [60]:
block_matrix = np.zeros((3, model.ncol))
block_matrix[0,np.arange(0,10)] = 1
block_matrix[1,np.arange(10,20)] = 1
block_matrix[2,np.arange(20,30)] = 1


In [61]:
pd.DataFrame(block_matrix)

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [62]:
block_matrix.shape[1]

30

In [63]:
block_constraints_2 = UBayconstraint(rho=np.array([1]), 
                             constraint_types=["max_size", "must_link"], 
                             constraint_vars=[1,[1,2]], 
                             num_elements=3,
                                block_matrix = block_matrix)

In [64]:
# define block weights

def build_block_weights(blocks, weights):
    weights_ass = []
    for i in blocks:
        weights_ass.append(weights[i])
    return np.array(weights_ass)

In [65]:
prior_weights = build_block_weights(blocks = np.repeat([0,1,2], 10), weights=np.array([0.5,1,2]))

## Build model and add constraints

In [66]:
model = UBaymodel(data=data,
                 target = labels,
                 feat_names = data.columns,
                 weights = prior_weights,
                 nr_features = 10)

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 67.16it/s]


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 69.09it/s]


In [67]:
pd.DataFrame(block_constraints.A)

,0,1,2
0,1.0,1.0,1.0


In [68]:
pd.DataFrame(block_constraints_2.A)

,0,1,2
0,1.0,1.0,1.0
1,0.0,1.0,-1.0
2,0.0,-1.0,1.0


In [69]:
model.setConstraints(block_constraints)
model.setConstraints(block_constraints_2, append=True)

# add max-size constraint
model.setConstraints(UBayconstraint(rho=np.array([1]), 
                             constraint_types=["max_size"], 
                             constraint_vars=[3], 
                             num_elements=model.ncol),
                    append=True)

In [70]:
model.getConstraints()

{0: {'A': array([[ 1.,  1.,  1.],
         [ 1.,  1.,  1.],
         [ 0.,  1., -1.],
         [ 0., -1.,  1.]]),
  'b': array([1., 1., 0., 0.]),
  'rho': array([1., 1., 1., 1.]),
  'block_matrix': array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1.,
          1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])},
 1: {'A': array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
          1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]]),
  'b': array([3.]),
  'rho': array([1.]),
  'block_matrix': array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0.

In [71]:
model.train()

(                         0
 mean.radius              0
 mean.texture             0
 mean.perimeter           1
 mean.area                0
 mean.smoothness          0
 mean.compactness         0
 mean.concavity           1
 mean.concave.points      1
 mean.symmetry            0
 mean.fractal.dimension   0
 radius.error             0
 texture.error            0
 perimeter.error          0
 area.error               0
 smoothness.error         0
 compactness.error        0
 concavity.error          0
 concave.points.error     0
 symmetry.error           0
 fractal.dimension.error  0
 worst.radius             0
 worst.texture            0
 worst.perimeter          0
 worst.area               0
 worst.smoothness         0
 worst.compactness        0
 worst.concavity          0
 worst.concave.points     0
 worst.symmetry           0
 worst.fractal.dimension  0,
 ['mean.perimeter', 'mean.concavity', 'mean.concave.points'])

In [72]:
model.counts

mean.radius                100
mean.texture                 0
mean.perimeter             100
mean.area                   99
mean.smoothness              0
mean.compactness             0
mean.concavity             100
mean.concave.points        100
mean.symmetry                0
mean.fractal.dimension       0
radius.error                 0
texture.error                0
perimeter.error              0
area.error                   0
smoothness.error             0
compactness.error            0
concavity.error              0
concave.points.error         0
symmetry.error               0
fractal.dimension.error      0
worst.radius               100
worst.texture                0
worst.perimeter            100
worst.area                 100
worst.smoothness             0
worst.compactness            1
worst.concavity            100
worst.concave.points       100
worst.symmetry               0
worst.fractal.dimension      0
dtype: object